In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset,Dataset
# use process bar tool
from tqdm import tqdm
from PIL import Image


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-final-dataset/dataset/test/adults/20.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/6.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/76.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/71.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/5.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/8.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/84.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/85.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/67.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/82.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/30.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/10.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/0.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/62.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/61.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/73.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/60.jpg
/kaggle/input/ml-f

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [3]:
aug_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=10), 
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Ensure the image is in RGB mode
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
def list_image_paths(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

def generate_augmented_dataset(original_dataset, num_augmented_images_per_class, transform):
    augmented_images = []
    augmented_labels = []
    for label in [0, 1]:
        counter = 0
        while counter < num_augmented_images_per_class:
            for img_path, lbl in zip(original_dataset.image_paths, original_dataset.labels):
                if lbl == label:
                    image = Image.open(img_path).convert("RGB")
                    augmented_img = transform(image)
                    augmented_images.append(augmented_img)
                    augmented_labels.append(lbl)
                    counter += 1
                    if counter >= num_augmented_images_per_class:
                        break
    return augmented_images, augmented_labels

# List image paths for each class
class_0_image_paths = list_image_paths('/kaggle/input/ml-final-dataset/dataset/train/adults')
class_1_image_paths = list_image_paths('/kaggle/input/ml-final-dataset/dataset/train/children')
class_0_labels = [0] * len(class_0_image_paths)
class_1_labels = [1] * len(class_1_image_paths)

# Combine both classes' image paths and labels
all_image_paths = class_0_image_paths + class_1_image_paths
all_labels = class_0_labels + class_1_labels

# Create the original dataset
original_dataset = CustomDataset(image_paths=all_image_paths, labels=all_labels, transform=None)

# Number of augmented images generated per class
num_augmented_images_per_class = 2000
augmented_images, augmented_labels = generate_augmented_dataset(original_dataset, num_augmented_images_per_class, aug_transforms)

# Create a dataset from augmented images and labels
augmented_dataset = [(img, lbl) for img, lbl in zip(augmented_images, augmented_labels)]

# Custom dataset for augmented data
class AugmentedDataset(Dataset):
    def __init__(self, augmented_data):
        self.augmented_data = augmented_data

    def __len__(self):
        return len(self.augmented_data)

    def __getitem__(self, idx):
        image, label = self.augmented_data[idx]
        return image, label

# Create dataset and DataLoader
augmented_dataset = AugmentedDataset(augmented_dataset)
train_loader = DataLoader(augmented_dataset, batch_size=100, shuffle=True)

In [4]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
#train_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/test', transform=transform)

#train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [5]:
# Load pre-trained EfficientNet model
#model = models.efficientnet_b0(pretrained=True)
model = models.resnet18(pretrained=True)

#for param in model.features.parameters():
#    param.requires_grad = False
#model = torch.load('/kaggle/working/2024_06_15_trial.pt')

# Modify the final layer to match the number of classes in the final project
num_ftrs = model.fc.in_features
model.fc= nn.Linear(num_ftrs, 2)  # Binary classification (2 classes)

# Transfer model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s]


In [6]:
# Training loop
#num_epochs = 30
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Use tqdm to create a progress bar for the data loader
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch"):
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch 1/1: 100%|██████████| 40/40 [00:08<00:00,  4.69batch/s]

Epoch [1/1], Loss: 0.4275


In [7]:
# Evaluate the model on the test dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 80.83%


In [8]:
# Evaluate the model on the train dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Train Accuracy: {100 * correct / total:.2f}%')

Train Accuracy: 89.45%


In [9]:
%pip install torchprofile torchsummary

Note: you may need to restart the kernel to use updated packages.


In [10]:
from torchsummary import summary
import torchvision.models as models
from torchprofile import profile_macs
# Calculate the number of trainable parameters and all parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())

print('trainable_params = ', trainable_params)
print('all_params = ', all_params)

# Create a dummy input tensor with the same size as the input images
input_tensor = torch.randn(1, 3, 224, 224).to(device)

# Calculate FLOPs
flops = profile_macs(model, input_tensor)
print(f'FLOPs: {flops / 1e9:.2f} GFLOPs')  # Convert to GFLOPs (GigaFLOPs)

trainable_params =  11177538
all_params =  11177538
FLOPs: 1.82 GFLOPs


In [11]:
#torch.save(model, '/kaggle/working/ResNet18_pretrained.pt')

In [12]:
#print("Model Layers:")
#omodel = models.efficientnet_b0(pretrained=True)
for layer in model.children():
    print(layer)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fal